# Quickstart

In this guide, we discuss how to deploy and interact with a Ray cluster on Kubernetes.

This guide is available at the Ray GitHub repo as an [executable Jupyter notebook](https://github.com/ray-project/ray/blob/master/doc/source/cluster/kuberay/quickstart.ipynb).


## Preparation

### Install Kubectl

We will use kubectl to interact with Kubernetes. Find installation instructions at the [Kubernetes documentation](https://kubernetes.io/docs/tasks/tools/#kubectl).

### Access a Kubernetes cluster

We will need access to a Kubernetes cluster. There are two options here:
1. Configure access to a remote Kubernetes cluster
**OR**
2. Run the examples locally by [installing kind](https://kind.sigs.k8s.io/docs/user/quick-start/#installation). Start your [kind](https://kind.sigs.k8s.io/) cluster by running

In [ ]:
! kind create cluster

To run the example in this guide, make sure your Kubernetes cluster can accomodate
additional resource requests of 4 CPU and 2.5Gi memory. 

## Deploying the KubeRay operator

Deploy the KubeRay Operator by cloning the KubeRay repo and applying the relevant configuration files from the master branch.  

In [ ]:
%%bash
# After the KubeRay 0.3.0 release branch cut, this documentation will be updated to refer to 0.3.0 branch.
git clone https://github.com/ray-project/kuberay
# This creates the KubeRay operator and all of the resources it needs.
kubectl create -k ray-operator/config/default
# Note that we must use "kubectl create" in the above command."kubectl apply" will not work due to https://github.com/ray-project/kuberay/issues/271

Confirm that the operator is running in the namespace `ray-system`.

In [ ]:
! kubectl -n ray-system get pod --selector=app.kubernetes.io/component=kuberay-operator
# NAME                                READY   STATUS    RESTARTS   AGE
# kuberay-operator-557c6c8bcd-t9zkz   1/1     Running   0          XXs

### Namespace-scoped operator
Note that the above command deploys the operator at _Kubernetes cluster scope_; the operator will manage resources in all Kubernetes namespaces.
**If your use-case requires running the operator at namespaced-scope**, refer to [the instructions at the KubeRay docs](https://github.com/ray-project/kuberay#single-namespace-version).

## Deploying a Ray Cluster

Once the KubeRay operator is running, we are ready to deploy a Ray cluster by deploying a RayCluster Custom Resource:

In [ ]:
# This Ray cluster is named `raycluster-autoscaler` because it has the optional Ray Autoscaler support enabled.
kubectl apply -f kuberay/ray-operator/samples/ray-cluster.autoscaler.yaml

Once the RayCluster CR has been created, you can view it by running

In [ ]:
! kubectl get raycluster
# NAME                    AGE
# raycluster-autoscaler   XXs

In [ ]:
! kubectl get pods --selector=ray.io/cluster=raycluster-autoscaler
# NAME                                             READY   STATUS    RESTARTS   AGE
# raycluster-autoscaler-head-mlmmq                 2/2     Running   0          XXs
# raycluster-autoscaler-worker-small-group-bt46r   1/1     Running   0          XXs

We see a Ray head with two containers -- the Ray container and autoscaler sidecar. We also a Ray worker with its single Ray container.

## Interacting with a Ray Cluster

Now, let's interact with the Ray cluster we've deployed.

### Accessing the cluster with kubectl exec

The most straightforward way to experiment with your Ray cluster is to
exec directly into the head pod. First, identify your cluster's head pod:

In [ ]:
! kubectl get pods --selector=ray.io/cluster=raycluster-autoscaler --selector=ray.io/node-type=head\
 -o custom-columns=POD:metadata.name --no-headers
# raycluster-autoscaler-head-mlmmq

Now, we can run a Ray program on the Ray head pod. The Ray program in the next cell asks the autoscaler to scale the cluster to a total of 3 CPUs.

In [ ]:
# Substitute your output from the last cell in place of "raycluster-autoscaler-head-mlmmq"
! kubectl exec raycluster-autoscaler-head-mlmmq -it -c ray-head -- python -c\
 "import ray; ray.init(); ray.autoscaler.sdk.request_resources(num_cpus=3)"

### Autoscaling

The last command should have triggered Ray pod upscaling. To confirm the new worker pod is up, let's query the RayCluster's pods again:

In [ ]:
! kubectl get pod --selector=ray.io/cluster=raycluster-autoscaler
# NAME                                             READY   STATUS    RESTARTS   AGE
# raycluster-autoscaler-head-mlmmq                 2/2     Running   0          XXs
# raycluster-autoscaler-worker-small-group-bt46r   1/1     Running   0          XXs
# raycluster-autoscaler-worker-small-group-krgmp   1/1     Running   0          XXs 

To get a summary of your cluster's status, run `ray status` on your cluster's Ray head node.

In [ ]:
! kubectl exec raycluster-autoscaler-head-mlmmq -it -c ray-head -- ray status 
# ======== Autoscaler status: 2022-07-21 14:38:20.849465 ========

Alternatively, to examine the full autoscaling logs, fetch the stdout of the Ray head pod's autoscaler sidecar:

In [ ]:
# This command gets the last 20 lines of autoscaler logs.
! kubectl logs raycluster-autoscaler-head-mlmmq -c autoscaler | tail -n 20
# ======== Autoscaler status: 2022-07-21 14:28:17.686505 ========
# ...

### The Ray head service

The KubeRay operator configures a [Kubernetes service]() targeting the Ray head pod.

### Using Ray Job submission

Ray provides a [Job Submission API]() which can be used to 

### Viewing the Ray Dashboard

### Accessing the cluster using Ray Client

## Cleanup

### Deleting a Ray Cluster
To delete the Ray Cluster we deployed in this example, you can run either of the following commands.

In [ ]:
# Delete by reference to the RayCluster custom resource
! kubectl delete raycluster raycluster-autoscaler

OR

In [ ]:
# Delete by reference to the yaml file we used to define the RayCluster CR 
! kubectl delete -f raycluster-autoscaler.yaml 

Confirm that the Ray Cluster's pods are gone by running

In [ ]:
! kubectl get pods

Note that it may take several seconds for the Ray pods to be fully terminated.

### Deleting the KubeRay operator
In typical operation, the KubeRay operator should be left as a long-running process that manages many Ray clusters.
If you would like to delete the operator and associated resources, run

In [ ]:
! kubectl delete -k kuberay/ray-operator/config/default

### Deleting a local kind cluster
Finally, if you'd like to delete your local kind cluster, run

! kind delete cluster